<a href="https://colab.research.google.com/github/nuruzzaman99/Artificial-Intelligence/blob/main/Skin_lession_classification_clinical_img_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

#import data from drive

In [ ]:
if not os.path.exists("./release_v0-20210711T044822Z-001.zip?dl=0"):
  !wget https://www.dropbox.com/s/7iyklsmv23ywr3p/release_v0-20210711T044822Z-001.zip?dl=0
  !unzip /content/release_v0-20210711T044822Z-001.zip?dl=0

else:
  print("The folder exsist")

#Load data from CSV files

In [3]:
meta_data = pd.read_csv("/content/release_v0/meta/meta.csv")
test_index = pd.read_csv("/content/release_v0/meta/test_indexes.csv")
train_index = pd.read_csv("/content/release_v0/meta/train_indexes.csv")
valid_index = pd.read_csv("/content/release_v0/meta/valid_indexes.csv")

In [4]:
meta_data.head()

,case_num,diagnosis,seven_point_score,pigment_network,streaks,pigmentation,regression_structures,dots_and_globules,blue_whitish_veil,vascular_structures,level_of_diagnostic_difficulty,elevation,location,sex,management,clinic,derm,case_id,notes
0,1,basal cell carcinoma,0,absent,absent,absent,absent,absent,absent,arborizing,medium,nodular,abdomen,female,excision,NEL/NEL025.JPG,NEL/Nel026.jpg,NaN,NaN
1,2,basal cell carcinoma,1,absent,absent,absent,absent,irregular,absent,absent,low,palpable,head neck,female,excision,NEL/NEL027.JPG,NEL/Nel028.jpg,NaN,NaN
2,3,basal cell carcinoma,1,absent,absent,absent,absent,irregular,absent,arborizing,medium,palpable,head neck,female,excision,NEL/Nel032.jpg,NEL/Nel033.jpg,NaN,NaN
3,4,basal cell carcinoma,4,absent,absent,absent,blue areas,irregular,present,within regression,low,palpable,lower limbs,male,excision,NEL/NEL034.JPG,NEL/Nel035.jpg,NaN,NaN
4,5,basal cell carcinoma,1,absent,absent,diffuse irregular,absent,absent,absent,absent,high,palpable,upper limbs,female,excision,NEL/NEL036.JPG,NEL/Nel037.jpg,NaN,NaN


#Load clinical images path

In [16]:
clinic_img_path = meta_data['clinic']
test_clinic_img_path = clinic_img_path.iloc[test_index['indexes']]
train_clinic_img_path = clinic_img_path.iloc[train_index['indexes']]
valid_clinic_img_path = clinic_img_path.iloc[valid_index['indexes']]

#Load index number from separated files

In [6]:
path = meta_data
test_path = path.iloc[test_index['indexes']]
train_path = path.iloc[train_index['indexes']]
valid_path = path.iloc[valid_index['indexes']]

#Set lebel for classes

In [7]:
y_train = train_path['level_of_diagnostic_difficulty']
y_valid = valid_path['level_of_diagnostic_difficulty']
y_test = test_path['level_of_diagnostic_difficulty']

In [8]:
y_test.head()

6       high
8        low
15       low
34    medium
41      high
Name: level_of_diagnostic_difficulty, dtype: object

#Process lebels into one-hot-encoding

In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical, plot_model

In [10]:
label_encoder = LabelEncoder()
train = label_encoder.fit_transform(y_train)

y_train_hc = to_categorical(train, num_classes = 3)

#print(y_train_hc)

In [11]:
label_encoder = LabelEncoder()
test = label_encoder.fit_transform(y_test)

y_test_hc = to_categorical(test, num_classes = 3)

#print(y_test_hc)

In [12]:
label_encoder = LabelEncoder()
valid = label_encoder.fit_transform(y_valid)

y_valid_hc = to_categorical(valid, num_classes = 3)

#print(y_valid_hc)

#Resize images and load into array

In [61]:
import numpy as np
import cv2

In [67]:
images_path = 'release_v0/images/'
im_size = (400, 400)

In [106]:
train_clinic_img = []
for img_path in train_clinic_img_path:
  tr_img = cv2.imread(images_path+img_path)
  a=cv2.resize(tr_img, (im_size), interpolation = cv2.INTER_AREA)
  train_clinic_img.append(a)

test_clinic_img = []
for img_path in test_clinic_img_path:
  ts_img = cv2.imread(images_path+img_path)
  a=cv2.resize(ts_img, (im_size), interpolation = cv2.INTER_AREA)
  test_clinic_img.append(a)


valid_clinic_img = []
for img_path in valid_clinic_img_path:
  vl_img = cv2.imread(images_path+img_path)
  a=cv2.resize(vl_img, (im_size), interpolation = cv2.INTER_AREA)
  valid_clinic_img.append(a)

In [107]:
train_clinic_img = np.array(train_clinic_img)
print(train_clinic_img.shape)

test_clinic_img = np.array(test_clinic_img)
print(test_clinic_img.shape)

valid_clinic_img = np.array(valid_clinic_img)
print(valid_clinic_img.shape)

(413, 400, 400, 3)
(395, 400, 400, 3)
(203, 400, 400, 3)


#Normalize data

In [108]:
x_train = train_clinic_img / 255.0
x_test = test_clinic_img / 255.0
x_valid = valid_clinic_img / 255.0

#Define Loss and Activation function

In [109]:
import tensorflow as tf
import math

In [110]:
def my_loss(y_true,y_pred):
  ts = 1.0
  err = tf.abs(y_true - y_pred)
  is_sml_err = err < ts
  sm_err = tf.square(err) / 2.0
  big_err = ts*(err -(0.5 * ts))
  return tf.where(is_sml_err, sm_err, big_err)

In [111]:
tf.keras.optimizers.Adam(learning_rate=0.0001, 
                         beta_1=0.9, 
                         beta_2=0.999, 
                         epsilon=1e-07, 
                         amsgrad = False, 
                         name='Adam')

#Build model

In [112]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop

In [113]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(400, 400, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))

model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 398, 398, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 199, 199, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 197, 197, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 98, 98, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 96, 96, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 48, 48, 128)      0

In [114]:
model.compile(optimizer= 'Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [115]:
history = model.fit(x_train, y_train_hc, 
                    epochs = 5,
                    steps_per_epoch = 100,
                    batch_size = 1, 
                    validation_data = (x_valid, y_valid_hc))

Epoch 1/5
100/100 [==============================] - 84s 834ms/step - loss: 1.9545 - accuracy: 0.3700 - val_loss: 1.0968 - val_accuracy: 0.3842
Epoch 2/5
100/100 [==============================] - 82s 826ms/step - loss: 1.1428 - accuracy: 0.3900 - val_loss: 1.1142 - val_accuracy: 0.3498
Epoch 3/5
100/100 [==============================] - 82s 822ms/step - loss: 1.1006 - accuracy: 0.3700 - val_loss: 1.1147 - val_accuracy: 0.3300
Epoch 4/5
100/100 [==============================] - 82s 821ms/step - loss: 1.1121 - accuracy: 0.3500 - val_loss: 1.0934 - val_accuracy: 0.3547
Epoch 5/5
100/100 [==============================] - 82s 821ms/step - loss: 1.0900 - accuracy: 0.4200 - val_loss: 1.0938 - val_accuracy: 0.3547


#Test the model

In [116]:
from keras.models import load_model

In [ ]:
model = load_model("/content/bestmodel")

acc = model.evaluate(x_test, y_test_hc, verbose=0)[1]

print(f"The accurecy of this model is {acc*100} %")

The accurecy of this model is 65.0632917881012 %


In [117]:
result = model.evaluate(x_test, y_test_hc)
print("Test accurecy: ", result[1]*100, "%")

13/13 [==============================] - 31s 2s/step - loss: 1.0803 - accuracy: 0.3949
Test accurecy:  39.49367105960846 %
